In [1]:
from SnakeBoard import SnakeBoard
from SnakeGame import SnakeGame
from NeuralNetwork import NeuralNetwork
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle

pygame 2.6.1 (SDL 2.28.4, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# ---------- User defined parameters ----------
# Miscellaneous parameters
restore_weights_prev_training = 0
manual_play = 0 # Get user input (keyboard) instead of neural network auto-play
show_visuals = 1 # Show the games (1) or just play and calculated in the back-end (0)
t_between_gen = 0 # Time (secs) between generations
n_gens_2_save_weights = 5 # Num of generations elapsed to save weights in a file

# Training parameters
n_of_gens = 5000 # Number of training generations
n_games_per_gen = 10 # Number of parallel games per generation
selected_games_per_gen = 2 # Selected baselines per generation to be used as references for mutations

# Mutation parameters
mrate_bias, mrate_weights = 0.05, 0.05
msize_bias, msize_weights = 0.2, 0.2

In [3]:
# ---------- Machine Learning main logic ---------- 
# Restore weights from previous training if required
if restore_weights_prev_training == 1:
    with open('./trainingHistory.bin', 'rb') as file:
        fileDataLoaded = pickle.load(file)
        file.close()

# Create 'N' games + their ANN instances
record_score, record_w_score = 0, 0
s_games, s_ann = [] , []
for idx in range(n_games_per_gen):
    s_games.append(SnakeGame())
    s_ann.append(NeuralNetwork())
    if restore_weights_prev_training == 1:
        s_ann[idx].set_weights_biases(fileDataLoaded["ann_weights_history"][-1][idx].weights,
                                      fileDataLoaded["ann_weights_history"][-1][idx].biases)
s_board = SnakeBoard(s_games, show_visuals)

# Create history of the best scores and ANN weights
game_status_history, ann_weights_history = list(), list()
if restore_weights_prev_training == 1:
    game_status_history = fileDataLoaded["game_status_history"]
    ann_weights_history = fileDataLoaded["ann_weights_history"]

s_board.init_board()

# Run number of generations
for idx_gen in range(n_of_gens):
    # 1 Step all games in current generation (until all games are over)
    while True:     
        game_status = list() 
        # 1_1 Go into each game to step individually
        for idx_game, game in enumerate(s_games):
            #1_1_1 Get current game state and decide the next move
            state = game.get_game_state()
            
            if show_visuals:
                next_move = game.get_key()            
            
            if manual_play == 1 and next_move == "PAUSE":
                while 1:
                    next_move = game.get_key()
                    if next_move != "IDLE":
                        break                    
            else:    
                next_move = s_ann[idx_game].calculate(state) 

            #1_1_2 Step game instance based on ANN calc. next move 
            [game_over, w_score, score] = game.step_game(next_move)

            #1_1_3 Save game data in game status dictionary array
            game_status.append({"game_over":    game_over, 
                                "score":        score, 
                                "w_score":      w_score, 
                                "idx_game":     idx_game})       

        # 1_2 Update graphics of all games (visual feedback)
        s_board.clear_board()
        s_board.update_board_elements()

        # 1_3 If all game instances are over, finish current generation
        if np.min([g["game_over"] for g in game_status])==True:
            break
    
    # 2 Get best scores and ANN weights in prev. generation + save history for the last generation
    game_status.sort(key=lambda x:x["w_score"],reverse =True) # Sort from best to worst game
    game_status_history.append(list(o.copy() for o in game_status)) # Game status history copy
    ann_weights_history.append(list(o.copy() for o in s_ann)) # ANN weights history copy

    # 3 If best score in curr. gen is an all-time record, save it
    #if game_status[0]["w_score"] > record_w_score: # Based on weighed score
    if game_status[0]["score"] > record_score: # Based on normal score
        record_w_score = game_status[0]["w_score"]
        record_score = game_status[0]["score"]
    
    # 4 Get the best "selected_games_per_gen" games in the current generation
    # and place them in the first positions
    for i in range(selected_games_per_gen):
        s_ann[i] = s_ann[game_status[i]["idx_game"]].copy()
    
    # 5 Mutate the best ones in the subsequent positions
    for i in range(selected_games_per_gen, n_games_per_gen):
        s_ann[i] = s_ann[np.mod(i,selected_games_per_gen)].copy()
        s_ann[i].mutate(mrate_weights, msize_weights, mrate_bias, msize_bias) # random mutations

    # 6 Reset all games
    for idx_game, game in enumerate(s_games):
        game.reset_game()

    # 7 Save weights in an external file when applicable
    if np.mod(idx_gen,n_gens_2_save_weights)==0:
        with open('./trainingHistory.bin', 'wb') as file:
            fileData = {"game_status_history": game_status_history, 
                        "ann_weights_history": ann_weights_history}
            pickle.dump(fileData, file)
            file.close()
    
    print("GEN ", idx_gen, " --- BEST SCORE: ", game_status[0]["score"] , " / ", 
            game_status[0]["w_score"] , " --- RECORD: ", record_score, " / ", record_w_score)
    time.sleep(t_between_gen)

s_board.quit_board()

SnakeBoard instance created.


NameError: name 'state2' is not defined